In [2]:
import json
import ast

from pyspark.mllib.linalg.distributed import RowMatrix

from pyspark.sql import Row
from pyspark.sql import SparkSession 

from pyspark.mllib.linalg import Vectors

import tqdm

from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.linalg import SparseVector

from scipy.sparse import find

import matplotlib.pyplot as plt
import numpy as np

from collections import Counter

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

NUMCLASTERS = 8

In [3]:
spark = SparkSession \
       .builder \
       .master('yarn') \
       .enableHiveSupport() \
       .getOrCreate()

#.master('yarn') \

In [1]:
df_lst = spark.read.table('prod.announcement_lst')
#df_lst.printSchema()

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [5]:
import math
from numpy import array
DICT = {'bedRent': 1,
        'commercialLandRent': 2,
        'dailyHouseRent': 3,
        'flatRent': 4,
        'flatSale': 5,
        'freeAppointmentObjectRent': 6,
        'officeRent': 7,
        'roomRent': 8,
        'shoppingAreaRent': 9,
        'warehouseSale': 10,
        'cottageSale': 11,
        'flatShareSale': 12,
        'freeAppointmentObjectSale': 13,
        'garageRent': 14,
        'garageSale': 15,
        'industryRent': 16,
        'newBuildingFlatSale': 17,
        'townhouseRent': 18,
        'buildingSale': 19,
        'cottageRent': 20,
        'dailyFlatRent': 21,
        'houseRent': 22,
        'houseSale': 23,
        'landSale': 24,
        'roomSale': 25,
        'warehouseRent': 26,
        'buildingRent': 27,
        'shoppingAreaSale': 28,
        'townhouseSale': 29,
        'businessRent': 30,
        'businessSale': 31,
        'dailyBedRent': 32,
        'dailyRoomRent': 33,
        'houseShareRent': 34,
        'houseShareSale': 35,
        'industrySale': 36,
        'officeSale': 37,
        'commercialLandSale': 38}

In [6]:
def lambda1(x):
    if x['priceType'] == 'squareMeter':
        if (x['price'] is None) or (x['totalarea'] is None):
            totalprice = 0
        else:
            totalprice = x['price'] * x['totalarea']
    elif x['price'] is None:
        totalprice = 0
    else: 
        totalprice = x['price']
    
    if x['floornumber'] is None:
        floornumber = 0
    else:
        floornumber = x['floornumber']
    
    if x['floorscount'] is None:
        floorscount = 0
    else:
        floorscount = x['floorscount']
    
    if x['category'] is None:
        category = 0
    else:
        category = DICT[x['category']] * 2
    
    if x['roomscount'] is None:
        roomscount = 0
    else:
        roomscount = x['roomscount']
    
    if x['totalarea'] is None:
        totalarea = 0
    else:
        totalarea = x['totalarea']
    
    return np.array([x['announcementid'],
                     floornumber, 
                     floorscount, 
                     category, 
                     roomscount, 
                     totalarea, 
                     totalprice])

In [7]:
rddSoprWithId = df_lst.select('announcementid', 
                         'floornumber', 
                         'floorscount', 
                         'category', 
                         'roomscount', 
                         'totalarea', 
                         'price',
                         'priceType') \
                 .dropDuplicates() \
                 .rdd \
                 .map(lambda x: lambda1(x)) #.show()
X = rddSoprWithId.map(lambda x: x[1:]).randomSplit([(2918553 - 1000)/2918553, 1000/2918553])[1]
#rdd_lst = df_lst.limit(10000).rdd#.show()

In [8]:
model = KMeans.train(X,
                     NUMCLASTERS, 
                     initializationMode="k-means||", 
                     initializationSteps=5, 
                     epsilon=1e-4)

#model.save("target/org/apache/spark/PythonKMeansExample/KMeansModel")

In [ ]:
#model = KMeansModel.load(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")

In [9]:
def addpredict(x):
    p = model.predict(x[1:])
    return (int(x[0]), p)

In [10]:
rddSoprWithIdPred = rddSoprWithId.map(lambda x: addpredict(x))

In [11]:
DICT_W_FOR_PAGE_TYPE = {"Card" : 3,
                        "CardJK" : 2,
                        "Listing" : 1,
                        "ListingFavorites" : 5}

DICT_W_FOR_EVENT_TYPE = {"card_show" : 3,
                        "phone_show" : 10}


data = [Row(page_type='Card', event_type='card_show', value = DICT_W_FOR_PAGE_TYPE["Card"] * DICT_W_FOR_EVENT_TYPE["card_show"]), 
        Row(page_type='CardJK', event_type='card_show', value = DICT_W_FOR_PAGE_TYPE["CardJK"] * DICT_W_FOR_EVENT_TYPE["card_show"]), 
        Row(page_type='Listing', event_type='card_show', value = DICT_W_FOR_PAGE_TYPE["Listing"] * DICT_W_FOR_EVENT_TYPE["card_show"]), 
        Row(page_type='ListingFavorites', event_type='card_show', value = DICT_W_FOR_PAGE_TYPE["ListingFavorites"] * DICT_W_FOR_EVENT_TYPE["card_show"]), 
        Row(page_type='Card', event_type='phone_show', value = DICT_W_FOR_PAGE_TYPE["Card"] * DICT_W_FOR_EVENT_TYPE["phone_show"]), 
        Row(page_type='CardJK', event_type='phone_show', value = DICT_W_FOR_PAGE_TYPE["CardJK"] * DICT_W_FOR_EVENT_TYPE["phone_show"]), 
        Row(page_type='Listing', event_type='phone_show', value = DICT_W_FOR_PAGE_TYPE["Listing"] * DICT_W_FOR_EVENT_TYPE["phone_show"]), 
        Row(page_type='ListingFavorites', event_type='phone_show', value = DICT_W_FOR_PAGE_TYPE["ListingFavorites"] * DICT_W_FOR_EVENT_TYPE["phone_show"])] 

dfdict = spark.createDataFrame(sc.parallelize(data))
dfdict.createOrReplaceTempView("dfdict")

In [12]:
#df_sopr.createOrReplaceTempView("df_sopr")

sqlDF = spark.sql("""select distinct a.offer_num, a.offer_id, b.user_num, b.user_id, c.value
                    from (select user_id, offer_id, value
                            from dfdict as a, prod.mles_sopr as b 
                            where a.page_type = b.page_type and a.event_type = b.event_type) as c
                         INNER JOIN (select row_number() OVER (ORDER BY a.user_id) as user_num, a.user_id 
                            from (select distinct user_id from prod.mles_sopr) as a) as b on c.user_id = b.user_id
                         INNER JOIN (select row_number() OVER (ORDER BY a.offer_id) as offer_num, a.offer_id 
                            from (select distinct offer_id from prod.mles_sopr limit 200000) as a) as a on c.offer_id = a.offer_id 
                         where b.user_id != 'noid'
                  """)

In [13]:
rdd = sqlDF.rdd.map(lambda x: (x['offer_id'], x)).join(rddSoprWithIdPred)

In [14]:
listrdd = [spark.createDataFrame(rdd.filter(lambda x: x[1][1] == i).map(lambda x: x[1][0])) for i in range(NUMCLASTERS)]

In [15]:
import os
from multiprocessing import Pool

In [16]:
def PoolALS(x):
    als = ALS(maxIter=5, regParam=0.01, userCol="user_num", itemCol="offer_id", ratingCol="value",
          coldStartStrategy="drop")
    model = als.fit(x)
    return model

In [20]:
modelList = []
with tqdm.tqdm(total=NUMCLASTERS) as progress:
    for i in range(NUMCLASTERS):
        modelList.append(PoolALS(listrdd[i]))
        progress.update(1)

100%|██████████| 8/8 [08:08<00:00, 57.00s/it] 
